In [ ]:
from typing import Literal, TypedDict, Union
import json

Starfish = TypedDict("Starfish", {
    "star_shaped": Literal[True],
    "fins": Literal[0],
    "name": str,
    "on a rock": bool
})

Jellyfish = TypedDict("Jellyfish", {
    "star_shaped": Literal[False],
    "fins": Literal[None],
    "name": str
})

Fish = TypedDict("Fish", {
    "star_shaped": Literal[False],
    "fins": int,
    "name": str
})

# Here, the `type | type` syntax could be used, but in the case of a tagged union it is more semantically explicit to wrap them as a singular unit
SeaCreature = Union[Starfish, Jellyfish, Fish]

json_obj = '''
{
    ...unknown
}
'''

sea_creature: SeaCreature = json.loads(json_obj)

if sea_creature["star_shaped"] == True: #! Note that if you wrote `if sea_creature["star_shaped"]: ` it wouldn't work!!!
    fins = sea_creature["fins"] # Known as `Literal[0]` 👍

if sea_creature["star_shaped"] == False:
    fins = sea_creature['fins'] # Known as `int | None`, no suggestion for 'on a rock' 👍

In [ ]:
from typing import Tuple

NotMissing =  Tuple[Literal[False], Literal[None], Literal["ignore"]] # True and None and 'ignore'
Missing = Tuple[Literal[True], int, Literal["div", "span", "a"] ] # True and int and tag name
Result = Union[Missing, NotMissing]

def missing_element(input: int) -> Result:
    # Do some logic related to DOM
    if input % 2:
        return (False, None, "ignore")
    else:
        if input % 4:
            return (True, input, "div") 
        elif (input / 4 == 1):
            return (True, input, "span")
        else:
            return (True, 2, "ignore") #❌ 'ignore' is the only suggestion, ✅ but does produce the correct error-lint behavior if used

(missing, id, tag_name) = missing_element(32)
if missing == True:
    id_ = id #❌ Should be id_: int
    tag = tag_name == "ignore" #❌ Should be tag: Literal[False]

result: Missing | NotMissing = missing_element(29)
result = missing_element(29)
if result[0] == True:
    id_ = result[1] # ✅
    tag = result[2] == "ignore" # ✅ => `Condition will always evaluate to False since the types "Literal['div', 'span', 'a']" and "Literal['ignore']" have no overlapPylancereportUnnecessaryComparison`
    tag = result[2] == "div" # ✅ autocomplete does not offer 'ignore'


In [ ]:
from typing import Tuple,  NamedTuple

class NotMissing(NamedTuple):
    missing: Literal[False]
    id: Literal[None]
    tag: Literal["ignore"]

class Missing(NamedTuple):
    missing: Literal[True]
    id: int
    tag: Literal["div", "span", "a"]

Result = Union[Missing, NotMissing]

def missing_element(input: int) -> Result:
    # Do some logic related to DOM
    if input % 2:
        return NotMissing(False, None, "ignore")
    else:
        if input % 4:
            return Missing(True, input, "div") 
        elif (input / 4 == 1):
            return Missing(missing=True, id=input, tag="span")
        else:
            return Missing(missing=True, id= 2, tag="ignore") #❌ 'ignore' is the only suggestion, ✅ but does produce the correct error-lint behavior if used

(missing, id, tag_name) = missing_element(32)
if missing == True:
    id_ = id #❌ Should be id_: int
    tag = tag_name == "ignore" #❌ Should be tag: Literal[False]

result: Missing | NotMissing = missing_element(29)
result = missing_element(29)
if result.missing == True:
    id_ = result.id # ✅
    tag = result.tag == "ignore" # ✅ => `Condition will always evaluate to False since the types "Literal['div', 'span', 'a']" and "Literal['ignore']" have no overlapPylancereportUnnecessaryComparison`
    tag = result.tag == "div" # ✅ autocomplete does not offer 'ignore'
